# Predicting on CENSO samples.

In [1]:
startyr = 2020
endyr = 2021

In [2]:
# Load modules
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from IPython.core.display import display, HTML

import pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import datetime as dt


In [3]:
# Column names
y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
          'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
          'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']


x_cols = ['IX_TOT', 'P02', 'P03', 'AGLO_rk', 'Reg_rk', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

x_cols1 = x_cols
predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']

x_cols2 = x_cols1 + predecir1
predecir2 = ['INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']

x_cols3 = x_cols2 + predecir2
# La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
# predecir3 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
predecir3 = ['PP07G1','PP07G_59', 'PP07I', 'PP07J', 'PP07K']

# Columnas de ingresos. Necesitan una regresion...
columnas_pesos = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']
# P21: INGRESO DE LA OCUPACIÓN PRINCIPAL 
# P47T: TOTAL (laboral y no laboral)
# PP08D1: Sueldos, jornales, etc.
# TOT_P12: TOTAL DE INGRESO  POR  OTRAS OCUPACIONES (Secundaria, previa...)
# T_VI: TOTAL DE INGRESOS NO LABORALES
# V12_M:  CUOTAS DE ALIMENTOS O AYUDA EN DINERO  DE PERSONAS QUE NO VIVEN EN EL HOGAR 
# V2_M: ingreso por JUBILACION O PENSION
# V3_M: INDEMNIZACION POR DESPIDO 
# V5_M: SUBSIDIO  O AYUDA SOCIAL DEL GOBIERNO, IGLESIAS, ETC.

x_cols4 = x_cols3 + predecir3
# Columnas de ingresos. Necesitan una regresion...
predecir4 = columnas_pesos
y_cols4 = predecir4

### Load info

In [4]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
##############################################################################################################
##############################################################################################################
##############################################################################################################

AGLO_rk = pd.read_csv('./../data/info/AGLO_rk')
Reg_rk = pd.read_csv('./../data/info/Reg_rk')

In [5]:
frac = 0.02

In [6]:
desemp_ratio = pd.read_csv('./../data/info/unemp_ratio_censo2010.csv', index_col=0)
desemp_ratio.index = pd.DatetimeIndex(desemp_ratio.index)

# Predicting

## Classification 1
Columnas ['CAT_OCUP', 'CAT_INAC', 'CH07']

In [7]:
# for yr in [str(s) for s in range(startyr, endyr)]:
#     print(yr)
# #     file_ = './../data/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv'
#     file_ = '/media/miglesia/Elements/CENSO_dirs/extracted_/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv'


#     X_censo = pd.read_csv(file_, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID', 'HOGAR_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
#            'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
#            'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0)
    
#     DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()
#     X_censo = X_censo.merge(DPTO_Region)
# #     print(X_censo.merge(radio_ref[['RADIO_REF_ID', 'PROV']], how = 'left')['PROV'].nunique())
#     X_censo = X_censo.merge(AGLO_rk[['AGLOMERADO', 'AGLO_rk']]).merge(Reg_rk[['Region', 'Reg_rk']])
    
    
#     ### Adaptacion del nivel de desempleo
    
    
    # Cargar modelo...
#     filename = './../fitted_RF/clf1_'+str(frac)+'_'+yr+'_ARG.sav'
#     clf1 = pickle.load(open(filename, 'rb'))
    
#     y_out1 = clf1.predict(X_censo[x_cols1].values)
#     y_censo_fit1 = pd.DataFrame(y_out1, index = X_censo.index, columns=predecir1)
#     Xy1_censo = pd.concat([X_censo, y_censo_fit1], axis = 1)

#     # save 
#     Xy1_censo.to_csv('./../data/yr_samples/RFC1_'+str(frac)+'_'+yr+'_ARG.csv', index = False)


In [8]:
# # Desempleo segun condicion de agro
  
#     X_censo_ = X_censo.copy()
#     X_censo_['AGLOMERADO'] = X_censo_['AGLOMERADO'] != 0

#     # X_censo_.groupby(['AGLOMERADO', 'CONDACT']).size().unstack()

#     ### Nivel Nacional:
#     nrs = X_censo_.groupby(['CONDACT', 'AGLOMERADO']).size().unstack()
#     display(np.round(100* nrs.loc[2] / (nrs.loc[1] + nrs.loc[2]), 3)) #.sort_values()

#     # nrs

In [9]:
if not os.path.exists('./../data/yr_samples'):
    os.makedirs('./../data/yr_samples')

In [10]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
        
#     file_ = './../data/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv'
    file_ = './../../yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv'


    X_censo = pd.read_csv(file_, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID', 'HOGAR_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
           'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
           'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0)
    
    DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()
    X_censo = X_censo.merge(DPTO_Region)
#     print(X_censo.merge(radio_ref[['RADIO_REF_ID', 'PROV']], how = 'left')['PROV'].nunique())
    X_censo = X_censo.merge(AGLO_rk[['AGLOMERADO', 'AGLO_rk']]).merge(Reg_rk[['Region', 'Reg_rk']])
    

    ## Tratamiento trimestral 
    
#     qs = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv', usecols=['Q'])
    qs = qs = pd.date_range(start = dt.datetime(int(yr),3,31), end = dt.datetime(int(yr),12,31), freq='Q').values
    
    desemp_ARG_censo = 1174037/(18645609 + 1174037)
    CONDACT_cnts = X_censo.CONDACT.value_counts()

    for q in qs:
        X_q = X_censo.copy()
        X_q['Q'] = q

        # desemp_ARG_censo_sample = CONDACT_cnts.loc[2]/(CONDACT_cnts.loc[1] + CONDACT_cnts.loc[2])
        
        ratio = desemp_ratio.loc[q].censo2010_ratio
        n_desempleados_ = ratio*(CONDACT_cnts[1] + CONDACT_cnts[2])*desemp_ARG_censo
        desemp_adic = round(n_desempleados_ - CONDACT_cnts.loc[2]) # Desempleados adicionales
        print(q)
        print(desemp_adic)

        if desemp_adic > 0:
            X_q.loc[
                X_q.query('CONDACT == 1').sample(desemp_adic).index,
                'CONDACT'
            ] = 2
        elif desemp_adic < 0:
            X_q.loc[
                X_q.query('CONDACT == 2').sample(- desemp_adic).index,
                'CONDACT'
            ] = 1

        desempleo = X_q.CONDACT.value_counts().loc[2] / (X_q.CONDACT.value_counts().loc[1] + X_q.CONDACT.value_counts().loc[2])
        print(desempleo)

        #################################    #################################    #################################

        ## CLASIF 1

        fitted_data_file_1 = './../data/yr_samples/RFC1_'+str(frac)+'ARG'+str(q)[:10]+'.csv'
        if not os.path.exists(fitted_data_file_1):
            # Cargar Modelo
            filename = './../fitted_RF/clf1_'+str(frac)+'_'+yr+'_ARG.sav'
            clf1 = pickle.load(open(filename, 'rb'))

            y_out1 = clf1.predict(X_q[x_cols1].values)
            y_censo_fit1 = pd.DataFrame(y_out1, index = X_q.index, columns=predecir1)
            Xy1_censo = pd.concat([X_q, y_censo_fit1], axis = 1)

        #     # save 
            Xy1_censo.to_csv(fitted_data_file_1, index = False)


2020
2020-03-31T00:00:00.000000000
7953
0.07701168489188692
2020-06-30T00:00:00.000000000
16753
0.0970052778541422
2020-09-30T00:00:00.000000000
12249
0.0867721934561879
2020-12-31T00:00:00.000000000
10018
0.08170336324041613


In [11]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    qs = qs = pd.date_range(start = dt.datetime(int(yr),3,31), end = dt.datetime(int(yr),12,31), freq='Q').values

    for q in qs:
    ## CLASIF 2

        ## Cargar Modelo
        fitted_data_file_1 = './../data/yr_samples/RFC1_'+str(frac)+'ARG'+str(q)[:10]+'.csv'
        fitted_data_file_2 = './../data/yr_samples/RFC2_'+str(frac)+'ARG'+str(q)[:10]+'.csv'

        if not os.path.exists(fitted_data_file_2):
            Xy1_censo = pd.read_csv(fitted_data_file_1)
            
            filename = './../fitted_RF/clf2_'+str(frac)+'_'+yr+'_ARG.sav'
            clf2 = pickle.load(open(filename, 'rb'))

            y_out2 = clf2.predict(Xy1_censo[x_cols2].values)
            y_censo_fit2 = pd.DataFrame(y_out2, index = Xy1_censo.index, columns=predecir2)

            Xy2_censo = pd.concat([Xy1_censo, y_censo_fit2], axis = 1)

        # #     # # save
            Xy2_censo.to_csv(fitted_data_file_2, index = False)


2020


In [12]:
## CLASIF 3
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    qs = qs = pd.date_range(start = dt.datetime(int(yr),3,31), end = dt.datetime(int(yr),12,31), freq='Q').values

    for q in qs:
        ## Cargar Modelo
        
                ## Cargar Modelo
        fitted_data_file_2 = './../data/yr_samples/RFC2_'+str(frac)+'ARG'+str(q)[:10]+'.csv'
        fitted_data_file_3 = './../data/yr_samples/RFC3_'+str(frac)+'ARG'+str(q)[:10]+'.csv'        
        
        if not os.path.exists(fitted_data_file_3):
            
            Xy2_censo = pd.read_csv(fitted_data_file_2)
            
            filename = './../fitted_RF/clf3_'+str(frac)+'_'+yr+'_ARG.sav'
            clf3 = pickle.load(open(filename, 'rb'))

            y_out3 = clf3.predict(Xy2_censo[x_cols3].values)
            y_censo_fit3 = pd.DataFrame(y_out3, index = Xy2_censo.index, columns=predecir3)

            Xy3_censo = pd.concat([Xy2_censo, y_censo_fit3], axis = 1)

        #     # # save
            Xy3_censo.to_csv(fitted_data_file_3, index = False)


2020


In [32]:
# REGRESION
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    qs = pd.date_range(start = dt.datetime(int(yr),3,31), end = dt.datetime(int(yr),12,31), freq='Q').values
    for q in qs:
        fitted_data_file_3 = './../data/yr_samples/RFC3_'+str(frac)+'ARG'+str(q)[:10]+'.csv'
        fitted_data_file_r = './../data/yr_samples/RFReg_'+str(frac)+'ARG'+str(q)[:10]+'.csv'
                
        if not os.path.exists(fitted_data_file_r):

            Xy3_censo = pd.read_csv(fitted_data_file_3)

            filename = './../fitted_RF/clf4_'+str(frac)+'_'+str(q)[:10]+'_ARG.sav'
            clf4 = pickle.load(open(filename, 'rb'))

            y_out4 = clf4.predict(Xy3_censo[x_cols4].values)
            y_censo_fit4 = pd.DataFrame(y_out4, index = Xy3_censo.index, columns=predecir4)


            Xy4_censo = pd.concat([Xy3_censo, y_censo_fit4], axis = 1)
            Xy4_censo.to_csv(fitted_data_file_r, index = False)


2020


FileNotFoundError: [Errno 2] No such file or directory: './../fitted_RF/clf4_0.02_2020-12-31_ARG.sav'

In [ ]:
xx

In [ ]:
## Cargar Modelo
filename = '/media/miglesia/Elements/CENSO_dirs/Pobreza/fitted_RF/clf4_'+str(frac)+'_'+str(q)[:10]+'_ARG.sav'
clf4 = pickle.load(open(filename, 'rb'))

y_out4 = clf4.predict(Xy3_censo[x_cols4].values)
y_censo_fit4 = pd.DataFrame(y_out4, index = Xy3_censo.index, columns=predecir4)


Xy4_censo = pd.concat([Xy3_censo, y_censo_fit4], axis = 1)

In [ ]:
# Xy4_censo.head().columns
Xy4_censo['Q'] = q

In [ ]:
data = X_censo.merge(radio_ref, how = 'left').groupby(['AGLOMERADO', 'CONDACT']).size().unstack()
display(100*round(data[2]/(data[1] + data[2]), 3))

In [ ]:
cnts = Xy4_censo.CONDACT.value_counts()
cnts[2]/(cnts[1] + cnts[2])

In [ ]:
0.077/desemp_ARG_censo

In [ ]:
data = Xy4_censo.merge(radio_ref, how = 'left').groupby(['AGLOMERADO', 'CONDACT']).size().unstack()
display(100*round(data[2]/(data[1] + data[2]), 3))

In [ ]:
xx

## Classification 2
Columna INGRESO

In [ ]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    Xy1_censo = pd.read_csv('./../data/yr_samples/RFC1_'+str(frac)+'_'+yr+'_ARG.csv')
    
    ## Cargar Modelo
    filename = './../fitted_RF/clf2_'+str(frac)+'_'+yr+'_ARG.sav'
    clf2 = pickle.load(open(filename, 'rb'))
       
    y_out2 = clf2.predict(Xy1_censo[x_cols2].values)
    y_censo_fit2 = pd.DataFrame(y_out2, index = Xy1_censo.index, columns=predecir2)

    Xy2_censo = pd.concat([Xy1_censo, y_censo_fit2], axis = 1)

#     # # save
    Xy2_censo.to_csv('./../data/yr_samples/RFC2_'+str(frac)+'_'+yr+'_ARG.csv', index = False)

## Classification 3
Columnas ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']

In [ ]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    Xy2_censo = pd.read_csv('./../data/yr_samples/RFC2_'+str(frac)+'_'+yr+'_ARG.csv')
    
    ## Cargar Modelo
    filename = './../fitted_RF/clf3_'+str(frac)+'_'+yr+'_ARG.sav'
    clf3 = pickle.load(open(filename, 'rb'))
       
    y_out3 = clf3.predict(Xy2_censo[x_cols3].values)
    y_censo_fit3 = pd.DataFrame(y_out3, index = Xy2_censo.index, columns=predecir3)

    Xy3_censo = pd.concat([Xy2_censo, y_censo_fit3], axis = 1)

    # # save
    Xy3_censo.to_csv('./../data/yr_samples/RFC3_'+str(frac)+'_'+yr+'_ARG.csv', index = False)

## Regresion

In [ ]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    Xy3_censo = pd.read_csv('./../data/yr_samples/RFC3_'+str(frac)+'_'+yr+'_ARG.csv')
        
    qs = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv', usecols=['Q'])
    
    for q in qs.Q.unique():

        ## Cargar Modelo
        filename = './../fitted_RF/clf4_'+str(frac)+'_'+q+'_ARG.sav'
        clf4 = pickle.load(open(filename, 'rb'))

        y_out4 = clf4.predict(Xy3_censo[x_cols4].values)
        y_censo_fit4 = pd.DataFrame(y_out4, index = Xy3_censo.index, columns=predecir4)


        Xy4_censo = pd.concat([Xy3_censo, y_censo_fit4], axis = 1)
        Xy4_censo.to_csv('./../data/yr_samples/RFReg_'+str(frac)+'ARG'+str(q)+'.csv', index = False)
